In [27]:
from __future__ import print_function, division, absolute_import, unicode_literals

import numpy as np
import random
import math
import pickle
import os

import matlab.engine
from scipy.interpolate import interp1d
from scipy.io import loadmat


width, height, cnt = [100, 100, 20]

num_cells = cnt

nbins = 4
image = np.zeros((width, height, nbins))
labels = np.zeros([width, height], dtype=np.bool)

imgx, imgy = width, height
nx = []  # hold the x value
ny = []  # holds the y value
nr = np.zeros([nbins, cnt])  # low energy image
nr2 = []  # high energy image
label = []  # labels whether or not the cell has cartilage

target_SNR = 100
# Finding the singnal in water that will be the basis for the noise
signal = 3.1109 * 10**3
# Finding the target noise
sigma = signal / target_SNR
# Finding the signal strength that would give the correct poisson noise
correct_signal = (signal / sigma)**2.

D = loadmat("/home/jericho/Downloads/bin/pcd_planar_imaging/DES_radiography/DES_3.0/Y_ML_val.mat")

cal_values = D["Y_ML_val"][:,1:]

for i in range(num_cells/2):

    # choosing random place for the cell
    nx.append(random.randrange(imgx))
    ny.append(random.randrange(imgy))
    rr = random.randint(0, len(cal_values) - 1)
    pp = random.randint(0, len(cal_values[cal_values == 0.]) - 1)
    
    inds_0 = np.where(cal_values[0,:] == 0)[0]
    # making a random value for the cartilage
#     if i % 2 == 0:
#         nr[:, i] = cal_values[1:5, rr]
#         label.append(1)
#     else:
        nr[:, i] = cal_values[1:5, inds_0[pp]]
        label.append(0)

for y in range(imgy):
    for x in range(imgx):
        dmin = math.hypot(imgx - 1, imgy - 1)
        j = -1
        for i in range(num_cells):
            d = math.hypot(nx[i] - x, ny[i] - y)
            if d < dmin:
                dmin = d
                j = i

        # Generating some poisson noise\
        for kk in range(nbins):
            image[x, y,
                   kk] = np.log((np.exp(nr[kk,j])) + np.random.normal(0,np.sqrt(np.exp(nr[kk,j])*2500))/2500)


        labels[x, y] = label[j]
        

r_min = 1
r_max = 10
border = 10
sigma = 10

#image = np.ones((nx, ny, 1))
label = np.ones((width, height))
mask = np.zeros([width, height], dtype=np.bool)
for _ in range(cnt):
    a = np.random.randint(border, width-border)
    b = np.random.randint(border, height-border)
    r = np.random.randint(r_min, r_max)
    h = np.random.randint(1,2)

    y,x = np.ogrid[-a:width-a, -b:height-b]
    m = x*x + y*y <= r*r
    mask = np.logical_or(mask, m)

    image[m] = h
labels[mask] = 0

#image += np.random.normal(scale=sigma, size=image.shape)
image -= np.amin(image)
image /= np.amax(image)

        
plt.imshow(image[:,:,1])
plt.figure()
plt.imshow(labels)

In [2]:
matlab_reshape = np.reshape(image,[image.size],order='F').copy()
matlab_reshape = list(np.squeeze(matlab_reshape)).copy()
matlab_reshape = matlab.double(matlab_reshape)

eng = matlab.engine.start_matlab()
ys = eng.net_val(matlab_reshape)

im_cart = np.reshape(ys,[width,height],order='F').copy()

In [14]:
%matplotlib qt
import matplotlib.pyplot as plt

# plt.imshow(im_cart[:,:])
# plt.figure()
plt.imshow(image[:,:,1])
plt.figure()
plt.imshow(labels)

(1, 300)

In [46]:
np.size(matlab_reshape)

(1, 1048576)

In [47]:
512*512*4

1048576

In [48]:
1048576/262144

4.0

In [52]:
import scipy.io as sio
sio.savemat("images.mat", {"image":image})

In [5]:
im_cart = np.zeros([nx,ny,1])

TypeError: 'list' object cannot be interpreted as an integer

In [28]:
print(nx)

[71, 59, 71, 52, 12, 24, 71, 44, 72, 69, 85, 78, 89, 18, 19, 67, 95, 0, 96, 76]
